# <b><span style='color:#2ae4f5'>|</span> L-shape Algorithm <a id = "1" > </b> 

`Problem Decomposition`

- **Master Problem**: Solve the master problem, which involves the first stage decision variables.

- **Subproblems**: For each scenario, solve the subproblems using the fixed variables from the master problem.

`Iterative Process`

* **Step 1**: Solve the master problem to get candidate solutions for the first-stage variables.

* **Step 2**: Solve the subproblems for each scenario using the candidate solutions from the master problem.

* **Step 3**: Check for feasibility and optimality. If the subproblems provide cuts (constraints) that are violated by the master problem solution, add these cuts to the master problem.

* **Step 4**: Update the master problem with the new cuts and resolve.

* **Step 5**: Repeat steps 1-4 until no more cuts are added, indicating convergence to the optimal solution.

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>|</span></b> Exercise <a id = "2" ></b></p>
</div>

$min \ z = 100x_1 + 150x_2 +E_{\xi}[q_1y_1+q_2y_2]$

$s.t.$

$x_1 + x_2 \le 120$

$6y_1 + 10y_2 \le 60x_1$

$8y_1 + 5y_2 \le 80x_1$

$ 0 \le y_1 \le d_1$

$ 0 \le y_2 \le d_2$

$ x_1 \ge 40$

$ x_2 \ge 20$

In [2]:
import numpy as np
from pulp import *
import math

In [3]:
idx = [1, 2]
scenarios = [1, 2]
prob = {1: 0.4, 2: 0.6}
d = {1: {1: 500, 2: 300}, 2: {1: 100, 2: 300}}
q = {1: {1: -24, 2: -28}, 2:{1: -28, 2: -32}}

In [4]:
# Define Master problem
master = LpProblem('MasterProblem', sense=LpMinimize)
x = LpVariable.dicts("X", idx, lowBound=0, cat="continuous")
theta = LpVariable('Theta', cat="continuous")

master += 100*x[1] + 150*x[2] + theta
master += x[1] + x[2] <= 120
master += x[1] >= 40
master += x[2] >= 20

In [ ]:
# Define Subproblems
h = {}
for s in scenarios:
	sub = LpProblem('SubProblem', LpMinimize)
	y = LpVariable.dicts('Y', idx, lowBound= 0, cat=LpContinuous)
	sub += q[1][s]*y[1] + q[2][s]*y[2]
	sub += 6*y[1] + 10*y[2] <= 60 * x[1]
	sub += 8*y[1] + 5*y[2] <= 80 * x[2]
	sub += y[1] <= d[1][s]
	sub += y[2] <= d[2][s]
 
	h_tmp = []
	for name, constraint in sub.constraints.items():
		h_tmp.append(-constraint.toDict()['constant'])
	h[s] = h_tmp
 
h

In [ ]:
T = []
for name, constraint in sub.constraints.items():
    coeff_row = []
    for var in master.variables():
        if var != theta:
            coeff = constraint.get(var, 0)
            coeff_row.append(coeff)
    T.append(coeff_row)
    
T

In [ ]:
# Step 1: set r, s, v
r, s, v = (0, 0, 0)

# Step 2: Define Master problem
master = LpProblem('MasterProblem', sense=LpMinimize)
x = LpVariable.dicts("X", idx, lowBound=0, cat="continuous")
theta = LpVariable('Theta', cat="continuous")

master += 100*x[1] + 150*x[2] + theta
master += x[1] + x[2] <= 120
master += x[1] >= 40
master += x[2] >= 20

not_stop_condition = True
while not_stop_condition:
    v += 1
    master.solve(PULP_CBC_CMD(msg=False))
    
    print(f'Iteration {v}')
    print(f"Master Problem Objective Function: {value(master.objective)}")

    if v == 1:
        theta.varValue = -math.inf
    print(f"Theta: {theta.varValue}")
    
    
    
        
    e = []
    E = []
    simplex_multiplier = {}
    for s in scenarios:
        sub = LpProblem('SubProblem', LpMinimize)
        y = LpVariable.dicts('Y', idx, lowBound= 0, cat=LpContinuous)
        sub += q[1][s]*y[1] + q[2][s]*y[2]
        sub += 6*y[1] + 10*y[2] <= 0 + 60 * x[1].varValue
        sub += 8*y[1] + 5*y[2] <= 0 + 80 * x[2].varValue
        sub += y[1] <= d[1][s]
        sub += y[2] <= d[2][s]
        
        sub.solve(PULP_CBC_CMD(msg=False, keepFiles=False))
        
        pi = []
        for _, c in sub.constraints.items():
            pi.append(c.pi)
        simplex_multiplier[s] = pi

        e.append(prob[s] * np.array([simplex_multiplier[s]]) @ np.array(h[s]))
        
        E.append(prob[s] * np.array([simplex_multiplier[s]]) @ np.array(T))
     
    e_sum = sum(e)
    E_sum = sum(E)
    w = e_sum - E_sum @ np.array([x[1].varValue, x[2].varValue])
    if w > theta.varValue:
        s += 1
        master.addConstraint(list(E_sum @ np.array([x[1], x[2]]))[0] + theta >= e_sum[0])
        print("added optima1ity cut to step 1.")
        print('***'*15)
    else:
        print('***'*15)
        print('STOP CONDITION')
        print('***'*15)
        print(f"Master Problem Objective Function: {value(master.objective)}")
        print(f"x1: {x[1].varValue}, x2: {x[2].varValue}")
        not_stop_condition = False
        